In [ ]:
'''
La derniere étape de notre projet est d'utiliser les modèles de machine learning choisi pour essayer de prédire 
les données de 2022 puis de comparer les écarts avec le reel.
'''

In [1]:
# Étape 1 : Preprocessing et feature engineering de la base de donnée 2022 pour la rendre exploitable.

# Import des modules et fichiers.
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

df_2022_full=pd.read_csv('../data/world-hapiness-report-2022.csv',sep=";")

# Suppression de toutes les lignes non relatives à 2022.
df_2022=df_2022_full[(df_2022_full['year']==2022)]

# Ajout des continents.
df_continents=pd.read_csv('../data/pays&continents.csv',sep=';')
df_2022=df_2022.merge(right=df_continents,on='Country name',how='outer')
df_2022.insert(1,'Regional indicator',df_2022.pop('Regional indicator'))

# Suppression des valeurs manquantes.
df_2022=df_2022.dropna()

# Suppression de la colonne des années.
df_2022=df_2022.drop(['year'],axis=1)

# Copie du jeux de données.
df_2022_continents = df_2022.copy()

# Séparation des variables explicatives et de la variable nom de pays de la variable cible.
X_test2022_continents=df_2022_continents.drop(['Life Ladder', 'Country name'],axis=1)
y_test2022=df_2022_continents['Life Ladder']

#On encode les continents de X_2022
X_test2022_continents=pd.get_dummies(X_test2022_continents,columns=['Regional indicator'])

# Normalisation des données.
scaler=MinMaxScaler()
cols=['Log GDP per capita','Social support','Healthy life expectancy at birth','Freedom to make life choices',
      'Generosity','Perceptions of corruption','Positive affect','Negative affect']
X_test2022_continents[cols]=scaler.fit_transform(X_test2022_continents[cols])

# Suppression de la colonne "Regional indicator_Central and Eastern Europe" pour correspondre à l'encodage des données
# fait par le OneHotEncoding.
X_test2022_continents=X_test2022_continents.drop(['Regional indicator_Central and Eastern Europe'],axis=1)

# X_test2022_continents et y_test2022 sont alors respectivements les jeux d'entraînement et de test.

In [2]:
# Étape 2 : Récupération des jeux d'entraînement et de test générés dans les notebooks précédents (données de 2006 
# à 2021).

# Import des données.
X_train=pd.read_csv('../data/X_train.csv')
X_test=pd.read_csv('../data/X_test.csv')
y_train=pd.read_csv('../data/y_train.csv')
y_test=pd.read_csv('../data/y_test.csv')

# Concaténation.
X_train2021=pd.concat([X_train,X_test],axis=0)
y_train2021=pd.concat([y_train,y_test],axis=0)

# Transofrmation du jeu y en tableau python.
y_train2021=y_train2021['Life Ladder'].values

# Normalisation des données.
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
cols=['Log GDP per capita','Social support','Healthy life expectancy at birth','Freedom to make life choices',
      'Generosity', 'Perceptions of corruption','Positive affect','Negative affect']
X_train2021[cols]=scaler.fit_transform(X_train2021[cols])

In [3]:
# Étape 3: Faire tourner le modele de Random forest pour la regression et le tester.

from sklearn.ensemble import RandomForestRegressor
rf_model = RandomForestRegressor(max_depth=30,max_features='sqrt',min_samples_leaf=1,min_samples_split=2,n_estimators=300)

rf_model.fit(X_train2021,y_train2021)

print("Score sur jeu d'entrainement 2021 :",rf_model.score(X_train2021,y_train2021))
print("Score sur jeu de test 2022 :",rf_model.score(X_test2022_continents,y_test2022))

Score sur jeu d'entrainement 2021 : 0.9857448604497061
Score sur jeu de test 2022 : 0.8030273312758086


In [4]:
# Réalisation d'une prédiction du Life Ladder de 2022 par pays grâce au modèle Random forest.
y_predic2022_continents = rf_model.predict(X_test2022_continents)

# Arrondissement de la prédictions à 2 décimales.
import numpy as np
y_predic2022_continents=np.round(y_predic2022_continents,2)

# Ajout d'une colonne avec les prédictions au jeu de données df_2022.
df_2022_continents['Predictions']=y_predic2022_continents
df_2022_continents=df_2022_continents.drop(cols,axis=1)

# Ajout d'une colonne pour représenter les variations en pourcentages.
df_2022_continents['Ecart en %']=np.round((df_2022_continents['Predictions']-df_2022_continents['Life Ladder'])/df_2022_continents['Life Ladder']*100,1)

In [5]:
df_2022_continents.head(10)

,Country name,Regional indicator,Life Ladder,Predictions,Ecart en %
1,Albania,Central and Eastern Europe,5.21,4.92,-5.6
2,Argentina,Latin America and Caribbean,6.26,6.13,-2.1
3,Armenia,Commonwealth of Independent States,5.38,5.00,-7.1
4,Australia,North America and ANZ,7.04,6.99,-0.7
5,Austria,Western Europe,7.00,6.79,-3.0
6,Bangladesh,South Asia,3.41,3.95,15.8
7,Belgium,Western Europe,6.86,7.03,2.5
8,Benin,Sub-Saharan Africa,4.22,3.84,-9.0
9,Bolivia,Latin America and Caribbean,5.93,5.27,-11.1
10,Botswana,Sub-Saharan Africa,3.44,4.67,35.8


In [6]:
df_2022_continents.drop('Country name',axis=1).groupby('Regional indicator').agg('mean').sort_values('Ecart en %',ascending=True)

# On constate que le modèle prédit plutot bien avec toutefois des écarts plus importants pour les valeurs extremes.

,Life Ladder,Predictions,Ecart en %
Regional indicator,,,
Southeast Asia,5.530000,5.075000,-7.400000
Latin America and Caribbean,6.211111,5.795556,-6.550000
Commonwealth of Independent States,5.592500,5.218750,-6.437500
East Asia,5.973333,5.596667,-6.433333
Central and Eastern Europe,6.044545,5.746364,-4.645455
North America and ANZ,6.907500,6.797500,-1.625000
Western Europe,6.893333,6.813333,-1.205556
South Asia,4.270000,4.146667,0.200000
Sub-Saharan Africa,4.289643,4.270714,2.017857


In [7]:
display(df_2022_continents.sort_values('Life Ladder',ascending=True).head(10))
display(df_2022_continents.sort_values('Life Ladder',ascending=True).tail(10))

,Country name,Regional indicator,Life Ladder,Predictions,Ecart en %
61,Lebanon,Middle East and North Africa,2.35,3.91,66.4
93,Sierra Leone,Sub-Saharan Africa,2.56,3.51,37.1
21,Congo (Kinshasa),Sub-Saharan Africa,3.21,4.08,27.1
113,Zimbabwe,Sub-Saharan Africa,3.30,4.08,23.6
66,Malawi,Sub-Saharan Africa,3.36,3.71,10.4
6,Bangladesh,South Asia,3.41,3.95,15.8
10,Botswana,Sub-Saharan Africa,3.44,4.67,35.8
19,Comoros,Sub-Saharan Africa,3.55,3.82,7.6
101,Tanzania,Sub-Saharan Africa,3.62,4.11,13.5
32,Ethiopia,Sub-Saharan Africa,3.63,4.37,20.4


,Country name,Regional indicator,Life Ladder,Predictions,Ecart en %
63,Lithuania,Central and Eastern Europe,7.04,5.89,-16.3
22,Costa Rica,Latin America and Caribbean,7.08,6.69,-5.5
64,Luxembourg,Western Europe,7.23,6.95,-3.9
83,Norway,Western Europe,7.30,7.43,1.8
78,Netherlands,Western Europe,7.39,7.08,-4.2
98,Sweden,Western Europe,7.43,7.48,0.7
45,Iceland,Western Europe,7.45,7.41,-0.5
26,Denmark,Western Europe,7.55,7.40,-2.0
50,Israel,Middle East and North Africa,7.66,6.79,-11.4
33,Finland,Western Europe,7.73,7.51,-2.8


In [8]:
display(df_2022_continents.sort_values('Ecart en %',ascending=False).head(10))
display(df_2022_continents.sort_values('Ecart en %',ascending=True).head(10))

,Country name,Regional indicator,Life Ladder,Predictions,Ecart en %
61,Lebanon,Middle East and North Africa,2.35,3.91,66.4
93,Sierra Leone,Sub-Saharan Africa,2.56,3.51,37.1
10,Botswana,Sub-Saharan Africa,3.44,4.67,35.8
21,Congo (Kinshasa),Sub-Saharan Africa,3.21,4.08,27.1
113,Zimbabwe,Sub-Saharan Africa,3.30,4.08,23.6
32,Ethiopia,Sub-Saharan Africa,3.63,4.37,20.4
6,Bangladesh,South Asia,3.41,3.95,15.8
101,Tanzania,Sub-Saharan Africa,3.62,4.11,13.5
39,Ghana,Sub-Saharan Africa,4.19,4.72,12.6
66,Malawi,Sub-Saharan Africa,3.36,3.71,10.4


,Country name,Regional indicator,Life Ladder,Predictions,Ecart en %
20,Congo (Brazzaville),Sub-Saharan Africa,5.81,4.38,-24.6
42,Guinea,Sub-Saharan Africa,5.32,4.16,-21.8
77,Nepal,South Asia,5.47,4.36,-20.3
80,Nicaragua,Latin America and Caribbean,6.39,5.33,-16.6
63,Lithuania,Central and Eastern Europe,7.04,5.89,-16.3
30,El Salvador,Latin America and Caribbean,6.49,5.44,-16.2
112,Vietnam,Southeast Asia,6.27,5.26,-16.1
73,Mongolia,East Asia,5.79,4.91,-15.2
71,Mexico,Latin America and Caribbean,7.04,6.00,-14.8
43,Honduras,Latin America and Caribbean,5.93,5.05,-14.8


In [9]:
'''
On pourrait se demander si lorsqu'il y a des valeurs un peu plus extremes, le continent n'entraîne pas le modele 
vers une réponse plus cohérente en fonction du continents. 
On réalise donc un essai sur les mêmes données amputées de la variable continent.
'''

"\nOn pourrait se demander si lorsqu'il y a des valeurs un peu plus extremes, le continent n'entraîne pas le modele \nvers une réponse plus cohérente en fonction du continents. \nOn réalise donc un essai sur les mêmes données amputées de la variable continent.\n"

In [10]:
# Séparation des variables explicatives de la variable cible et suppression des noms de pays.
X_test2022_no_continents=df_2022.drop(['Life Ladder','Country name','Regional indicator'],axis=1)
y_test2022_no_continents=df_2022['Life Ladder']

#On normalise les variables explicatives
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
cols=['Log GDP per capita','Social support','Healthy life expectancy at birth','Freedom to make life choices','Generosity',
      'Perceptions of corruption','Positive affect','Negative affect']
X_test2022_no_continents[cols]=scaler.fit_transform(X_test2022_no_continents[cols])

#X_test2022_no_continents et y_test2022_no_continents sont alors respectivements nos variables explicatives et notre variable cible de notre jeu de test

In [11]:
# Suppression variables des continents du jeu de données 2021.
X_train2021_no_continents=X_train2021.drop(["Regional indicator_Commonwealth of Independent States"
                              ,"Regional indicator_East Asia"
                              ,"Regional indicator_Latin America and Caribbean"
                              ,"Regional indicator_Middle East and North Africa"
                              ,"Regional indicator_North America and ANZ"
                              ,"Regional indicator_South Asia"
                              ,"Regional indicator_Southeast Asia"
                              ,"Regional indicator_Sub-Saharan Africa"
                              ,"Regional indicator_Western Europe"],axis=1)

#On normalise les variables explicatives
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
cols=['Log GDP per capita','Social support','Healthy life expectancy at birth','Freedom to make life choices','Generosity',
      'Perceptions of corruption','Positive affect','Negative affect']
X_train2021_no_continents[cols]=scaler.fit_transform(X_train2021_no_continents[cols])

In [12]:
from sklearn.ensemble import RandomForestRegressor
rf_model = RandomForestRegressor(max_depth=30,max_features='sqrt',min_samples_leaf=1,min_samples_split=2,n_estimators=300)

rf_model.fit(X_train2021_no_continents,y_train2021)

print("Score sur jeu d'entrainement 2021 :",rf_model.score(X_train2021_no_continents,y_train2021))
print("Score sur jeu de test 2022 :",rf_model.score(X_test2022_no_continents,y_test2022))

Score sur jeu d'entrainement 2021 : 0.9835026110935509
Score sur jeu de test 2022 : 0.7384410487638684


In [13]:
# Réalisation d'une prédiction du Life Ladder de 2022 par pays grâce au modèle Random forest.
y_predic2022_no_continents = rf_model.predict(X_test2022_no_continents)

# Arrondissement de la prédictions à 2 décimales.
import numpy as np
y_predic2022_no_continents=np.round(y_predic2022_no_continents,2)

# Copie du jeu de donnée 2022 et suppression des continents.
df_2022_no_continents = df_2022.copy()

# Ajout d'une colonne avec les prédictions au jeu de données df_2022.
df_2022_no_continents['Predictions']=y_predic2022_no_continents
df_2022_no_continents=df_2022_no_continents.drop(cols,axis=1)

# Ajout d'une colonne pour représenter les variations en pourcentages.
df_2022_no_continents['Ecart en %']=np.round((df_2022_no_continents['Predictions']-df_2022_no_continents['Life Ladder'])/df_2022_no_continents['Life Ladder']*100,1)

In [14]:
df_2022_no_continents

,Country name,Regional indicator,Life Ladder,Predictions,Ecart en %
1,Albania,Central and Eastern Europe,5.21,4.94,-5.2
2,Argentina,Latin America and Caribbean,6.26,6.04,-3.5
3,Armenia,Commonwealth of Independent States,5.38,4.92,-8.6
4,Australia,North America and ANZ,7.04,6.88,-2.3
5,Austria,Western Europe,7.00,6.69,-4.4
...,...,...,...,...,...
108,United States,North America and ANZ,6.69,6.14,-8.2
109,Uruguay,Latin America and Caribbean,6.67,6.29,-5.7
110,Uzbekistan,Commonwealth of Independent States,6.02,5.25,-12.8
112,Vietnam,Southeast Asia,6.27,5.21,-16.9


In [15]:
df_2022_no_continents.drop('Country name',axis=1).groupby('Regional indicator').agg('mean').sort_values('Ecart en %',ascending=True)

#Grâce à ce df on s'aperçoit que notre modele prédit plutot bien, il a un peu plus de mal a prédire les valeurs extremes

,Life Ladder,Predictions,Ecart en %
Regional indicator,,,
Latin America and Caribbean,6.211111,5.528889,-10.866667
Commonwealth of Independent States,5.592500,5.085000,-8.862500
Southeast Asia,5.530000,5.080000,-7.200000
East Asia,5.973333,5.576667,-6.766667
Central and Eastern Europe,6.044545,5.784545,-4.000000
North America and ANZ,6.907500,6.675000,-3.400000
Western Europe,6.893333,6.761111,-1.911111
South Asia,4.270000,4.173333,1.100000
Sub-Saharan Africa,4.289643,4.257143,1.589286


In [16]:
display(df_2022_no_continents.sort_values('Life Ladder',ascending=True).head(10))
display(df_2022_no_continents.sort_values('Life Ladder',ascending=True).tail(10))

,Country name,Regional indicator,Life Ladder,Predictions,Ecart en %
61,Lebanon,Middle East and North Africa,2.35,3.95,68.1
93,Sierra Leone,Sub-Saharan Africa,2.56,3.42,33.6
21,Congo (Kinshasa),Sub-Saharan Africa,3.21,4.02,25.2
113,Zimbabwe,Sub-Saharan Africa,3.30,4.13,25.2
66,Malawi,Sub-Saharan Africa,3.36,3.66,8.9
6,Bangladesh,South Asia,3.41,4.00,17.3
10,Botswana,Sub-Saharan Africa,3.44,4.62,34.3
19,Comoros,Sub-Saharan Africa,3.55,3.84,8.2
101,Tanzania,Sub-Saharan Africa,3.62,4.26,17.7
32,Ethiopia,Sub-Saharan Africa,3.63,4.26,17.4


,Country name,Regional indicator,Life Ladder,Predictions,Ecart en %
63,Lithuania,Central and Eastern Europe,7.04,5.91,-16.1
22,Costa Rica,Latin America and Caribbean,7.08,6.63,-6.4
64,Luxembourg,Western Europe,7.23,6.85,-5.3
83,Norway,Western Europe,7.30,7.39,1.2
78,Netherlands,Western Europe,7.39,7.03,-4.9
98,Sweden,Western Europe,7.43,7.32,-1.5
45,Iceland,Western Europe,7.45,7.40,-0.7
26,Denmark,Western Europe,7.55,7.28,-3.6
50,Israel,Middle East and North Africa,7.66,6.58,-14.1
33,Finland,Western Europe,7.73,7.40,-4.3


In [17]:
display(df_2022_no_continents.sort_values('Ecart en %',ascending=False).head(10))
display(df_2022_no_continents.sort_values('Ecart en %',ascending=True).head(10))

,Country name,Regional indicator,Life Ladder,Predictions,Ecart en %
61,Lebanon,Middle East and North Africa,2.35,3.95,68.1
10,Botswana,Sub-Saharan Africa,3.44,4.62,34.3
93,Sierra Leone,Sub-Saharan Africa,2.56,3.42,33.6
113,Zimbabwe,Sub-Saharan Africa,3.30,4.13,25.2
21,Congo (Kinshasa),Sub-Saharan Africa,3.21,4.02,25.2
101,Tanzania,Sub-Saharan Africa,3.62,4.26,17.7
32,Ethiopia,Sub-Saharan Africa,3.63,4.26,17.4
6,Bangladesh,South Asia,3.41,4.00,17.3
39,Ghana,Sub-Saharan Africa,4.19,4.77,13.8
55,Jordan,Middle East and North Africa,4.36,4.78,9.6


,Country name,Regional indicator,Life Ladder,Predictions,Ecart en %
71,Mexico,Latin America and Caribbean,7.04,5.30,-24.7
20,Congo (Brazzaville),Sub-Saharan Africa,5.81,4.40,-24.3
30,El Salvador,Latin America and Caribbean,6.49,4.98,-23.3
80,Nicaragua,Latin America and Caribbean,6.39,4.90,-23.3
77,Nepal,South Asia,5.47,4.30,-21.4
42,Guinea,Sub-Saharan Africa,5.32,4.22,-20.7
37,Georgia,Commonwealth of Independent States,5.29,4.25,-19.7
43,Honduras,Latin America and Caribbean,5.93,4.78,-19.4
41,Guatemala,Latin America and Caribbean,6.15,4.96,-19.3
9,Bolivia,Latin America and Caribbean,5.93,4.84,-18.4


In [18]:
# Pour terminer, on aggrège les deux jeux de données.
df_2022 = df_2022_continents
df_2022

,Country name,Regional indicator,Life Ladder,Predictions,Ecart en %
1,Albania,Central and Eastern Europe,5.21,4.92,-5.6
2,Argentina,Latin America and Caribbean,6.26,6.13,-2.1
3,Armenia,Commonwealth of Independent States,5.38,5.00,-7.1
4,Australia,North America and ANZ,7.04,6.99,-0.7
5,Austria,Western Europe,7.00,6.79,-3.0
...,...,...,...,...,...
108,United States,North America and ANZ,6.69,6.24,-6.7
109,Uruguay,Latin America and Caribbean,6.67,6.41,-3.9
110,Uzbekistan,Commonwealth of Independent States,6.02,5.40,-10.3
112,Vietnam,Southeast Asia,6.27,5.26,-16.1


In [19]:
# Changement de nom des colonnes
new_names =  {'Predictions' : 'Prédictions avec continents',
              'Ecart en %'  : 'Écart en % avec continents'}

df_2022 = df_2022.rename(new_names, axis = 1)
df_2022

,Country name,Regional indicator,Life Ladder,Prédictions avec continents,Écart en % avec continents
1,Albania,Central and Eastern Europe,5.21,4.92,-5.6
2,Argentina,Latin America and Caribbean,6.26,6.13,-2.1
3,Armenia,Commonwealth of Independent States,5.38,5.00,-7.1
4,Australia,North America and ANZ,7.04,6.99,-0.7
5,Austria,Western Europe,7.00,6.79,-3.0
...,...,...,...,...,...
108,United States,North America and ANZ,6.69,6.24,-6.7
109,Uruguay,Latin America and Caribbean,6.67,6.41,-3.9
110,Uzbekistan,Commonwealth of Independent States,6.02,5.40,-10.3
112,Vietnam,Southeast Asia,6.27,5.26,-16.1


In [20]:
df_2022['Prédictions sans continents'] = df_2022_no_continents['Predictions']
df_2022['Écart en % sans continents'] = df_2022_no_continents['Ecart en %']
df_2022

,Country name,Regional indicator,Life Ladder,Prédictions avec continents,Écart en % avec continents,Prédictions sans continents,Écart en % sans continents
1,Albania,Central and Eastern Europe,5.21,4.92,-5.6,4.94,-5.2
2,Argentina,Latin America and Caribbean,6.26,6.13,-2.1,6.04,-3.5
3,Armenia,Commonwealth of Independent States,5.38,5.00,-7.1,4.92,-8.6
4,Australia,North America and ANZ,7.04,6.99,-0.7,6.88,-2.3
5,Austria,Western Europe,7.00,6.79,-3.0,6.69,-4.4
...,...,...,...,...,...,...,...
108,United States,North America and ANZ,6.69,6.24,-6.7,6.14,-8.2
109,Uruguay,Latin America and Caribbean,6.67,6.41,-3.9,6.29,-5.7
110,Uzbekistan,Commonwealth of Independent States,6.02,5.40,-10.3,5.25,-12.8
112,Vietnam,Southeast Asia,6.27,5.26,-16.1,5.21,-16.9


In [21]:
# Ajout d'une colonne pour représenter les différences de pourcentages.
df_2022['Différences valeurs écarts'] = df_2022['Prédictions sans continents']-df_2022['Prédictions avec continents']
df_2022

,Country name,Regional indicator,Life Ladder,Prédictions avec continents,Écart en % avec continents,Prédictions sans continents,Écart en % sans continents,Différences valeurs écarts
1,Albania,Central and Eastern Europe,5.21,4.92,-5.6,4.94,-5.2,0.02
2,Argentina,Latin America and Caribbean,6.26,6.13,-2.1,6.04,-3.5,-0.09
3,Armenia,Commonwealth of Independent States,5.38,5.00,-7.1,4.92,-8.6,-0.08
4,Australia,North America and ANZ,7.04,6.99,-0.7,6.88,-2.3,-0.11
5,Austria,Western Europe,7.00,6.79,-3.0,6.69,-4.4,-0.10
...,...,...,...,...,...,...,...,...
108,United States,North America and ANZ,6.69,6.24,-6.7,6.14,-8.2,-0.10
109,Uruguay,Latin America and Caribbean,6.67,6.41,-3.9,6.29,-5.7,-0.12
110,Uzbekistan,Commonwealth of Independent States,6.02,5.40,-10.3,5.25,-12.8,-0.15
112,Vietnam,Southeast Asia,6.27,5.26,-16.1,5.21,-16.9,-0.05


In [22]:
# Ajout d'une colonne pour représenter les différences de pourcentages.
df_2022['% variations écarts'] = df_2022['Écart en % sans continents']-df_2022['Écart en % avec continents']
df_2022

,Country name,Regional indicator,Life Ladder,Prédictions avec continents,Écart en % avec continents,Prédictions sans continents,Écart en % sans continents,Différences valeurs écarts,% variations écarts
1,Albania,Central and Eastern Europe,5.21,4.92,-5.6,4.94,-5.2,0.02,0.4
2,Argentina,Latin America and Caribbean,6.26,6.13,-2.1,6.04,-3.5,-0.09,-1.4
3,Armenia,Commonwealth of Independent States,5.38,5.00,-7.1,4.92,-8.6,-0.08,-1.5
4,Australia,North America and ANZ,7.04,6.99,-0.7,6.88,-2.3,-0.11,-1.6
5,Austria,Western Europe,7.00,6.79,-3.0,6.69,-4.4,-0.10,-1.4
...,...,...,...,...,...,...,...,...,...
108,United States,North America and ANZ,6.69,6.24,-6.7,6.14,-8.2,-0.10,-1.5
109,Uruguay,Latin America and Caribbean,6.67,6.41,-3.9,6.29,-5.7,-0.12,-1.8
110,Uzbekistan,Commonwealth of Independent States,6.02,5.40,-10.3,5.25,-12.8,-0.15,-2.5
112,Vietnam,Southeast Asia,6.27,5.26,-16.1,5.21,-16.9,-0.05,-0.8


In [23]:
df_2022.describe()

,Life Ladder,Prédictions avec continents,Écart en % avec continents,Prédictions sans continents,Écart en % sans continents,Différences valeurs écarts,% variations écarts
count,103.000000,103.000000,103.000000,103.000000,103.000000,103.000000,103.000000
mean,5.591359,5.408058,-1.803883,5.337961,-2.936893,-0.070097,-1.133010
std,1.229449,1.069843,12.422331,1.051158,13.152231,0.156177,2.689354
min,2.350000,3.510000,-24.600000,3.420000,-24.700000,-0.700000,-10.900000
25%,4.675000,4.480000,-8.950000,4.400000,-9.200000,-0.110000,-1.850000
50%,5.890000,5.400000,-3.000000,5.250000,-4.700000,-0.050000,-0.900000
75%,6.465000,6.195000,1.250000,6.145000,0.900000,0.020000,0.500000
max,7.730000,7.510000,66.400000,7.400000,68.100000,0.290000,5.000000


In [24]:
# On constate que les prédictions sont moins précises si l'on enlève les données relatives aux continents.

In [25]:
# Séparation des écarts en positifs et négatifs
ecarts_positifs = df_2022[df_2022['Écart en % avec continents'] > 0]['Écart en % avec continents']
ecarts_negatifs = df_2022[df_2022['Écart en % avec continents'] < 0]['Écart en % avec continents']

# Calcul de la moyenne et de la médiane pour les écarts positifs
ecart_positif_moyen = ecarts_positifs.mean()
ecart_positif_median = ecarts_positifs.median()

# Calcul de la moyenne et de la médiane pour les écarts négatifs
ecart_negatif_moyen = ecarts_negatifs.mean()
ecart_negatif_median = ecarts_negatifs.median()

# Affichage des résultats
print("Écart négatif moyen:", ecart_negatif_moyen)
print("Écart positif moyen:", ecart_positif_moyen)
print("Écart négatif médian:", ecart_negatif_median)
print("Écart positif médian:", ecart_positif_median)

Écart négatif moyen: -7.368055555555555
Écart positif moyen: 11.49
Écart négatif médian: -5.95
Écart positif médian: 6.25


In [26]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Calcul de l'erreur quadratique moyenne (MSE) et de la racine carrée de l'erreur quadratique moyenne (RMSE)
# avzec les continents.
mse_continents = mean_squared_error(y_test2022, y_predic2022_continents)
rmse_continents = np.sqrt(mse_continents)

# Affichage des résultats
print(f"Erreur quadratique moyenne avec les continents (MSE) : {mse_continents}")
print(f"Racine carrée de l'erreur quadratique moyenne avec les continents (RMSE) : {rmse_continents}")

# Calcul de l'erreur quadratique moyenne (MSE) et de la racine carrée de l'erreur quadratique moyenne (RMSE) 
# sans les continents.
mse_no_continents = mean_squared_error(y_test2022, y_predic2022_no_continents)
rmse_no_continents = np.sqrt(mse_no_continents)

# Affichage des résultats
print(f"Erreur quadratique moyenne sans les continents (MSE) : {mse_no_continents}")
print(f"Racine carrée de l'erreur quadratique moyenne sans les continents (RMSE) : {rmse_no_continents}")

Erreur quadratique moyenne avec les continents (MSE) : 0.2953417475728155
Racine carrée de l'erreur quadratique moyenne avec les continents (RMSE) : 0.5434535376394338
Erreur quadratique moyenne sans les continents (MSE) : 0.3910485436893203
Racine carrée de l'erreur quadratique moyenne sans les continents (RMSE) : 0.6253387431539168


In [27]:
# On constate que la RMSE augmente sans les continents. Le modèles est donc bien moins performant.

In [29]:
from joblib import dump, load

dump(rf_model, '../models/happinessmeter.joblib')

['../models/happinessmeter.joblib']